In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
train = pd.read_csv('Train.csv')
test = pd.read_csv('Test.csv')

print(train.shape, test.shape)

(13645, 22) (8745, 20)


# feature preprocessing

In [3]:
train.head()

,EmpID,EmpName,LanguageOfCommunication,Age,Gender,JobProfileIDApplyingFor,HighestDegree,DegreeBranch,GraduatingInstitute,LatestDegreeCGPA,...,CurrentCTC,ExpectedCTC,MartialStatus,EmpScore,CurrentDesignation,CurrentCompanyType,DepartmentInCompany,TotalLeavesTaken,BiasInfluentialFactor,FitmentPercent
0,11041,John,English,35,Male,JR85289,B.Tech,Electrical,Tier 1,7,...,21,26,Married,5,SSE,Enterprise,Design,20,YearsOfExperince,95.40
1,15079,William,English,26,Male,JR87525,B.Tech,Artificial Intelligence,Tier 3,7,...,15,19,Married,5,BA,MidSized,Engineering,6,NaN,67.09
2,18638,James,English,36,Female,JR87525,PhD,Computer Science,Tier 1,6,...,15,24,Single,5,SDE,MidSized,Engineering,19,Gender,91.26
3,3941,Charles,English,29,Female,JR87525,BCA,Information Technology,Tier 2,5,...,16,24,Married,5,SDE,Startup,Product,16,Gender,72.29
4,5936,George,English,25,Male,JR70175,Dual M.Tech,Computer Science,Tier 3,8,...,24,32,Married,5,SDE,Enterprise,Engineering,10,DegreeBranch,86.34


In [4]:
train.columns

Index(['EmpID', 'EmpName', 'LanguageOfCommunication', 'Age', 'Gender',
       'JobProfileIDApplyingFor', 'HighestDegree', 'DegreeBranch',
       'GraduatingInstitute', 'LatestDegreeCGPA', 'YearsOfExperince',
       'GraduationYear', 'CurrentCTC', 'ExpectedCTC', 'MartialStatus',
       'EmpScore', 'CurrentDesignation', 'CurrentCompanyType',
       'DepartmentInCompany', 'TotalLeavesTaken', 'BiasInfluentialFactor',
       'FitmentPercent'],
      dtype='object')

In [5]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 13645 entries, 0 to 13644
Data columns (total 22 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   EmpID                    13645 non-null  int64  
 1   EmpName                  13645 non-null  object 
 2   LanguageOfCommunication  13645 non-null  object 
 3   Age                      13645 non-null  int64  
 4   Gender                   13645 non-null  object 
 5   JobProfileIDApplyingFor  13645 non-null  object 
 6   HighestDegree            13645 non-null  object 
 7   DegreeBranch             13645 non-null  object 
 8   GraduatingInstitute      13645 non-null  object 
 9   LatestDegreeCGPA         13645 non-null  int64  
 10  YearsOfExperince         13645 non-null  int64  
 11  GraduationYear           13645 non-null  int64  
 12  CurrentCTC               13645 non-null  int64  
 13  ExpectedCTC              13645 non-null  int64  
 14  MartialStatus         

In [3]:
print(train.shape)

train1 = train[[ 'LanguageOfCommunication', 'Age', 'Gender',
       'JobProfileIDApplyingFor', 'HighestDegree', 'DegreeBranch',
       'GraduatingInstitute', 'LatestDegreeCGPA', 'YearsOfExperince',
       'CurrentCTC', 'MartialStatus',
       'EmpScore', 'CurrentDesignation', 'CurrentCompanyType',
       'DepartmentInCompany', 'TotalLeavesTaken']].copy()
print(train1.shape)

train1['GraduatingInstitute'] = train['GraduatingInstitute'].map({'Tier 1':3,'Tier 2':2, 'Tier 3':1})
train1['ExpectedHike'] = (train['ExpectedCTC']-train['CurrentCTC'])/train['CurrentCTC']*100
print(train1.shape)
train1.head()

(13645, 22)
(13645, 16)
(13645, 17)


,LanguageOfCommunication,Age,Gender,JobProfileIDApplyingFor,HighestDegree,DegreeBranch,GraduatingInstitute,LatestDegreeCGPA,YearsOfExperince,CurrentCTC,MartialStatus,EmpScore,CurrentDesignation,CurrentCompanyType,DepartmentInCompany,TotalLeavesTaken,ExpectedHike
0,English,35,Male,JR85289,B.Tech,Electrical,3,7,12,21,Married,5,SSE,Enterprise,Design,20,23.809524
1,English,26,Male,JR87525,B.Tech,Artificial Intelligence,1,7,3,15,Married,5,BA,MidSized,Engineering,6,26.666667
2,English,36,Female,JR87525,PhD,Computer Science,3,6,6,15,Single,5,SDE,MidSized,Engineering,19,60.000000
3,English,29,Female,JR87525,BCA,Information Technology,2,5,6,16,Married,5,SDE,Startup,Product,16,50.000000
4,English,25,Male,JR70175,Dual M.Tech,Computer Science,1,8,2,24,Married,5,SDE,Enterprise,Engineering,10,33.333333


In [4]:
target_col = train['BiasInfluentialFactor'].fillna('Unknown')
target_col.head()

0    YearsOfExperince
1             Unknown
2              Gender
3              Gender
4        DegreeBranch
Name: BiasInfluentialFactor, dtype: object

# Creating X and Y

In [5]:
X = pd.get_dummies(train1, drop_first=True).values
y = target_col.copy().values
print(X.shape,y.shape)

(13645, 46) (13645,)


# encoding label

In [6]:
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_en = le.fit_transform(y)

In [7]:
le.classes_

array(['CurrentCompanyType', 'DegreeBranch', 'EmpScore', 'Ethinicity',
       'Gender', 'HighestDegree', 'LatestDegreeCGPA', 'MartialStatus',
       'Unknown', 'YearsOfExperince'], dtype=object)

# Model

In [8]:
from lightgbm import LGBMClassifier
from sklearn.model_selection import cross_val_score, StratifiedKFold
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import time

# lgb

# With cross val

In [64]:
class_weight={6: 250,
 3: 500,
 0: 250,
 7:100,
 8:90
 }

In [65]:
clf_lgb = LGBMClassifier(verbose=2, n_estimators=3000, objective='multiclass',
                         n_jobs=-1)

In [66]:
n_splits = 5
skf = StratifiedKFold(n_splits=n_splits)
ls_accuracy=[]
ls_scores=[]
ls_confusion_mat=[]

start=time.time()
for ind_train, ind_val in skf.split(X,y_en):
    X_train = X[ind_train,:]
    y_train = y_en[ind_train,]
    X_val = X[ind_val,:]
    y_val = y_en[ind_val,]
    
    print('---------------------------------------\nfitting...')
    clf_lgb.fit(X_train,y_train, early_stopping_rounds=300,
               eval_set=[(X_val, y_val)])
    
    print('predicting...')
    y_pred = clf_lgb.predict(X_val)
    
    ls_accuracy.append(accuracy_score(y_val,y_pred))
    ls_scores.append(classification_report(y_val,y_pred))
    ls_confusion_mat.append(classification_report(y_val,y_pred))
stop=time.time()
print(f'\n\ntime: {round((stop-start),2)} s')
print(f'acc: {ls_accuracy}')
print(f'Mean accuracy: {np.mean(ls_accuracy)}')
print(f'Std accuracy: {np.std(ls_accuracy)}')

---------------------------------------
fitting...
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.160682
[LightGBM] [Debug] init for col-wise cost 0.000026 seconds, init for row-wise cost 0.002849 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.007330 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Dense Multi-Val Bin
[LightGBM] [Info] Total Bins 298
[LightGBM] [Info] Number of data points in the train set: 10916, number of used features: 46
[LightGBM] [Info] Start training from score -2.453169
[LightGBM] [Info] Start training from score -1.562989
[LightGBM] [Info] Start training from score -3.028889
[LightGBM] [Info] Start training from score -3.729640
[LightGBM] [Info] Start training from score -1.655940
[LightGBM] [Info] Start training from score -3.155947
[LightGBM] [Info] Start training from score -3.9

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[25]	valid_0's multi_logloss: 0.560825
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[26]	valid_0's multi_logloss: 0.557736
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 1

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[39]	valid_0's multi_logloss: 0.54342
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with 

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[54]	valid_0's multi_logloss: 0.543432
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree wit

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[68]	valid_0's multi_logloss: 0.549184
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with

...

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[295]	valid_0's multi_logloss: 0.82163
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 18 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tre

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[308]	valid_0's multi_logloss: 0.837031
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 15 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[309]	valid_0's multi_logloss: 0.838187
[LightGBM] [Debug] Trained a tree with leaves = 31 and de

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 14 and depth = 7
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[322]	valid_0's multi_logloss: 0.85566
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Warning] No further 

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Debug] Trained a tree with leaves = 13 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[335]	valid_0's multi_logloss: 0.873491
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tr

predicting...


time: 61.88 s
acc: [0.7984609747160132, 0.7929644558446317, 0.7907658482960791, 0.7936973250274826, 0.7944301942103335]
Mean accuracy: 0.794063759618908
Std accuracy: 0.0025174883162063398


In [67]:
for i in ls_scores:
    print(i)

              precision    recall  f1-score   support

           0       0.69      0.27      0.39       235
           1       0.91      0.92      0.92       571
           2       0.78      0.95      0.86       131
           3       0.07      0.02      0.02        66
           4       0.87      0.99      0.92       521
           5       0.93      0.91      0.92       117
           6       0.34      0.25      0.29        52
           7       0.56      0.64      0.60        56
           8       0.68      0.73      0.70       667
           9       0.87      0.96      0.91       313

    accuracy                           0.80      2729
   macro avg       0.67      0.67      0.65      2729
weighted avg       0.78      0.80      0.78      2729

              precision    recall  f1-score   support

           0       0.65      0.32      0.43       235
           1       0.89      0.93      0.91       571
           2       0.79      0.95      0.86       132
           3       0.15 

# With smote

# Data Split

In [9]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train,y_val = train_test_split(X,y_en, test_size=.2, stratify=y_en,random_state=123)
print(X_train.shape, X_val.shape, y_train.shape,y_val.shape)

(10916, 46) (2729, 46) (10916,) (2729,)


# smote

In [69]:
[(c,i) for i,c in enumerate(list(train1.columns))]

[('LanguageOfCommunication', 0),
 ('Age', 1),
 ('Gender', 2),
 ('JobProfileIDApplyingFor', 3),
 ('HighestDegree', 4),
 ('DegreeBranch', 5),
 ('GraduatingInstitute', 6),
 ('LatestDegreeCGPA', 7),
 ('YearsOfExperince', 8),
 ('CurrentCTC', 9),
 ('MartialStatus', 10),
 ('EmpScore', 11),
 ('CurrentDesignation', 12),
 ('CurrentCompanyType', 13),
 ('DepartmentInCompany', 14),
 ('TotalLeavesTaken', 15),
 ('ExpectedHike', 16)]

In [93]:
dict(pd.Series(y_train).value_counts())

{8: 2669,
 1: 2286,
 4: 2084,
 9: 1249,
 0: 939,
 2: 527,
 5: 466,
 3: 262,
 7: 224,
 6: 210}

In [82]:
sampling_strategy_dict={8: 2669,
 1: 2600,
 4: 2400,
 9: 1900,
 0: 1900,
 2: 1600,
 5: 1600,
 3: 2500,
 7: 1600,
 6: 1500}

In [10]:
sampling_strategy_dict={
 2: 700,
 5: 700,
 3: 500,
 7: 500,
 6: 500}

In [163]:
# categorical_features=np.array([0,2,3,4,5,6,10,12,13,14]),

In [11]:
from imblearn.over_sampling import SMOTE

sm = SMOTE(sampling_strategy=sampling_strategy_dict,
              random_state=123, n_jobs=-1)
X_train_res, y_train_res = sm.fit_resample(X_train, y_train)

print(X_train.shape,y_train.shape)
print(X_train_res.shape, y_train_res.shape)

(10916, 46) (10916,)
(12127, 46) (12127,)


In [12]:
pd.Series(y_train_res).value_counts()/len(X_train_res)*100

8    22.008741
1    18.850499
4    17.184794
9    10.299332
0     7.743053
5     5.772244
2     5.772244
7     4.123031
6     4.123031
3     4.123031
dtype: float64

In [13]:
clf_lgb = LGBMClassifier(verbose=2, n_estimators=2000, objective='multiclass',
                         n_jobs=-1)

In [14]:
clf_lgb.fit(X_train_res,y_train_res, early_stopping_rounds=200,
               eval_set=[(X_val, y_val)])

[LightGBM] [Debug] Dataset::GetMultiBinFromSparseFeatures: sparse rate 0.862575
[LightGBM] [Debug] Dataset::GetMultiBinFromAllFeatures: sparse rate 0.680824
[LightGBM] [Debug] init for col-wise cost 0.002495 seconds, init for row-wise cost 0.003475 seconds
[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.003646 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Debug] Using Sparse Multi-Val Bin
[LightGBM] [Info] Total Bins 6306
[LightGBM] [Info] Number of data points in the train set: 12127, number of used features: 46
[LightGBM] [Info] Start training from score -2.558374
[LightGBM] [Info] Start training from score -1.668631
[LightGBM] [Info] Start training from score -2.852109
[LightGBM] [Info] Start training from score -3.188582
[LightGBM] [Info] Start training from score -1.761145
[LightGBM] [Info] Start training from score -2.852109
[LightGBM] [Info]

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[11]	valid_0's multi_logloss: 0.702692
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with l

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[25]	valid_0's multi_logloss: 0.566361
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[26]	valid_0's multi_logloss: 0.564245
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 1

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[40]	valid_0's multi_logloss: 0.553551
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[41]	valid_0's multi_logloss: 0.553832

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[54]	valid_0's multi_logloss: 0.557699
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 16
[LightGBM] [Debug] Trained a tree wit

...

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[178]	valid_0's multi_logloss: 0.684046
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with l

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[195]	valid_0's multi_logloss: 0.703504
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[196]	valid_0's multi_logloss: 0.704563
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 19
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[212]	valid_0's multi_logloss: 0.723214
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 13
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 10
[213]	valid_0's multi_logloss: 0.724252

[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 8
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 17
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[228]	valid_0's multi_logloss: 0.742962
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 14
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 9
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 15
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 12
[LightGBM] [Debug] Trained a tree with leaves = 31 and depth = 11
[229]	valid_0's multi_logloss: 0.744225

LGBMClassifier(n_estimators=2000, objective='multiclass', verbose=2)

In [15]:
y_pred = clf_lgb.predict(X_val)

print(classification_report(y_val,y_pred))
print(confusion_matrix(y_val,y_pred))

              precision    recall  f1-score   support

           0       0.66      0.30      0.41       235
           1       0.90      0.92      0.91       572
           2       0.79      0.96      0.87       132
           3       0.29      0.03      0.05        66
           4       0.86      1.00      0.92       521
           5       0.90      0.97      0.93       116
           6       0.55      0.31      0.40        52
           7       0.63      0.79      0.70        56
           8       0.67      0.70      0.68       667
           9       0.88      0.96      0.92       312

    accuracy                           0.80      2729
   macro avg       0.71      0.69      0.68      2729
weighted avg       0.78      0.80      0.78      2729

[[ 70   3   0   0  21   1   1   0 135   4]
 [  5 524   0   0   0   0   1   0  42   0]
 [  0   0 127   3   0   0   1   0   1   0]
 [  0   0  26   2   0   0   2  10  26   0]
 [  1   0   0   0 520   0   0   0   0   0]
 [  0   0   0   0   0 113 

## Hyperopt

In [20]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

space={'max_depth': hp.quniform("max_depth", 5, 10, 1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.6,0.9),
        'min_child_weight' : hp.quniform('min_child_weight', 5, 50, 1),
        'learning_rate': hp.uniform("learning_rate",0.01,0.1),
        'subsample': hp.uniform("subsample",0.75,1),
        'reg_alpha' : hp.uniform("reg_alpha",0.0001,1),
        'reg_lambda' : hp.uniform("reg_lambda",0.0001,1),
    }

def objective(space):
    clf_lgm_pt = LGBMClassifier(n_jobs=-1,
                               objective='multiclass',
                               n_estimators=2000,
                               max_depth=int(space['max_depth']),
                               colsample_bytree=space['colsample_bytree'],
                               min_child_weight=int(space['min_child_weight']),
                               learning_rate=space['learning_rate'],
                               subsample=space['subsample'],
                                reg_alpha = space['reg_alpha'],
                                reg_lambda = space['reg_lambda'],
                               )
    
    print('Fitting...')
    clf_lgm_pt.fit(X_train_res,y_train_res, early_stopping_rounds=200,
               eval_set=[(X_val, y_val)])
    
    print('Predicting...')
    y_pred = clf_lgm_pt.predict(X_val)
    accuracy = accuracy_score(y_val,y_pred)
    
    print('accuracy: {}'.format(accuracy))
    return {'loss': -accuracy, 'status': STATUS_OK }

In [21]:
trials = Trials()


best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 30,
                        trials = trials)

Fitting...                                                                                                             
[1]	valid_0's multi_logloss: 1.77067                                                                                   
Training until validation scores don't improve for 200 rounds                                                          
[2]	valid_0's multi_logloss: 1.66071                                                                                   
[3]	valid_0's multi_logloss: 1.52415                                                                                   
[4]	valid_0's multi_logloss: 1.42158                                                                                   
[5]	valid_0's multi_logloss: 1.34682                                                                                   
[6]	valid_0's multi_logloss: 1.28182                                                                                   
[7]	valid_0's multi_logloss: 1.21029    

[67]	valid_0's multi_logloss: 0.554705                                                                                 
[68]	valid_0's multi_logloss: 0.553896                                                                                 
[69]	valid_0's multi_logloss: 0.552157                                                                                 
[70]	valid_0's multi_logloss: 0.55087                                                                                  
[71]	valid_0's multi_logloss: 0.550595                                                                                 
[72]	valid_0's multi_logloss: 0.550165                                                                                 
[73]	valid_0's multi_logloss: 0.54953                                                                                  
[74]	valid_0's multi_logloss: 0.549183                                                                                 
[75]	valid_0's multi_logloss: 0.548616  

[135]	valid_0's multi_logloss: 0.548429                                                                                
[136]	valid_0's multi_logloss: 0.548769                                                                                
[137]	valid_0's multi_logloss: 0.5489                                                                                  
[138]	valid_0's multi_logloss: 0.548691                                                                                
[139]	valid_0's multi_logloss: 0.54886                                                                                 
[140]	valid_0's multi_logloss: 0.549167                                                                                
[141]	valid_0's multi_logloss: 0.549236                                                                                
[142]	valid_0's multi_logloss: 0.549293                                                                                
[143]	valid_0's multi_logloss: 0.549887 

[203]	valid_0's multi_logloss: 0.559589                                                                                
[204]	valid_0's multi_logloss: 0.559822                                                                                
[205]	valid_0's multi_logloss: 0.560303                                                                                
[206]	valid_0's multi_logloss: 0.560679                                                                                
[207]	valid_0's multi_logloss: 0.560834                                                                                
[208]	valid_0's multi_logloss: 0.561065                                                                                
[209]	valid_0's multi_logloss: 0.56121                                                                                 
[210]	valid_0's multi_logloss: 0.561517                                                                                
[211]	valid_0's multi_logloss: 0.561667 

[271]	valid_0's multi_logloss: 0.571537                                                                                
[272]	valid_0's multi_logloss: 0.571629                                                                                
[273]	valid_0's multi_logloss: 0.571819                                                                                
[274]	valid_0's multi_logloss: 0.571841                                                                                
[275]	valid_0's multi_logloss: 0.572101                                                                                
[276]	valid_0's multi_logloss: 0.572369                                                                                
[277]	valid_0's multi_logloss: 0.572729                                                                                
[278]	valid_0's multi_logloss: 0.57281                                                                                 
[279]	valid_0's multi_logloss: 0.57312  

...

[42]	valid_0's multi_logloss: 0.642424                                                                                 
[43]	valid_0's multi_logloss: 0.637196                                                                                 
[44]	valid_0's multi_logloss: 0.633113                                                                                 
[45]	valid_0's multi_logloss: 0.628427                                                                                 
[46]	valid_0's multi_logloss: 0.624132                                                                                 
[47]	valid_0's multi_logloss: 0.620267                                                                                 
[48]	valid_0's multi_logloss: 0.61639                                                                                  
[49]	valid_0's multi_logloss: 0.612613                                                                                 
[50]	valid_0's multi_logloss: 0.609446  

[110]	valid_0's multi_logloss: 0.556331                                                                                
[111]	valid_0's multi_logloss: 0.556126                                                                                
[112]	valid_0's multi_logloss: 0.556151                                                                                
[113]	valid_0's multi_logloss: 0.556045                                                                                
[114]	valid_0's multi_logloss: 0.555779                                                                                
[115]	valid_0's multi_logloss: 0.555681                                                                                
[116]	valid_0's multi_logloss: 0.555723                                                                                
[117]	valid_0's multi_logloss: 0.555734                                                                                
[118]	valid_0's multi_logloss: 0.555663 

[178]	valid_0's multi_logloss: 0.560682                                                                                
[179]	valid_0's multi_logloss: 0.560745                                                                                
[180]	valid_0's multi_logloss: 0.560909                                                                                
[181]	valid_0's multi_logloss: 0.561006                                                                                
[182]	valid_0's multi_logloss: 0.561118                                                                                
[183]	valid_0's multi_logloss: 0.561198                                                                                
[184]	valid_0's multi_logloss: 0.561335                                                                                
[185]	valid_0's multi_logloss: 0.561384                                                                                
[186]	valid_0's multi_logloss: 0.561703 

[246]	valid_0's multi_logloss: 0.569239                                                                                
[247]	valid_0's multi_logloss: 0.569255                                                                                
[248]	valid_0's multi_logloss: 0.569468                                                                                
[249]	valid_0's multi_logloss: 0.569473                                                                                
[250]	valid_0's multi_logloss: 0.569666                                                                                
[251]	valid_0's multi_logloss: 0.569852                                                                                
[252]	valid_0's multi_logloss: 0.570077                                                                                
[253]	valid_0's multi_logloss: 0.570138                                                                                
[254]	valid_0's multi_logloss: 0.570332 

[314]	valid_0's multi_logloss: 0.577885                                                                                
[315]	valid_0's multi_logloss: 0.578042                                                                                
[316]	valid_0's multi_logloss: 0.578082                                                                                
[317]	valid_0's multi_logloss: 0.578156                                                                                
[318]	valid_0's multi_logloss: 0.578309                                                                                
[319]	valid_0's multi_logloss: 0.578405                                                                                
Early stopping, best iteration is:                                                                                     
[119]	valid_0's multi_logloss: 0.55553
Predicting...                                                                                                          
a

In [22]:
best_hyperparams

{'colsample_bytree': 0.8540023235526808,
 'learning_rate': 0.06661674024882186,
 'max_depth': 5.0,
 'min_child_weight': 6.0,
 'reg_alpha': 0.13840765929551507,
 'reg_lambda': 0.6424847575483579,
 'subsample': 0.9310185517239598}